In [19]:
import polars.selectors as cs
import numpy as np

import polars as pl

import matplotlib.pyplot as plt

In [20]:
def hist_of_numerical_values(df: pl.DataFrame):
    """
    Generates histograms for all numerical columns in a Polars DataFrame.

    Args:
        df: The Polars DataFrame.

    Returns:
        A dictionary where keys are column names and values are the histogram data.
    """
    histograms = {}
    for col_name in df.columns:
        if df[col_name].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]:
            hist_data = df[col_name].hist()
            histograms[col_name] = hist_data
    return histograms

In [21]:
index_cols = {
    1: ['l_returnflag', 'l_linestatus'],
    3: ['l_orderkey','o_orderdate','o_shippriority'],
    4: ['o_orderpriority'],
    5: ['n_name'],
    6: [],
    7: ['supp_nation', 'cust_nation', 'l_year'], 
    8: ['o_year'],
    9: ['nation', 'o_year'],
    12: ['l_shipmode'],
    13: ['c_count'],
    14: [],
    15: [],
    17: [],
    19: [],
    20: [],
    21: ['s_name'],
    22: ['cntrycode']
}
scale_required = set([1,4, 5, 6, 7, 9, 12, 13, 17, 19, 21, 22])
# not 8, 14
# skip 3, skip 20
queries_to_run = [1, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 17, 19, 21, 22]

In [28]:
# for query_ind in queries_to_run:
num_trials = 1000
avg_errors = {}
all_nulls = {}
for query_ind in queries_to_run:
#     print(query_ind)
#     1/128, 1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1.0, 2.0, 4.0
    num_nulls = 0.
    total_error = []
    for mi in [1/16]:
        null_info = {}
        orig = pl.read_csv(f'../unnoised/q{query_ind}.csv')
        if query_ind == 22:
            orig = orig.select(
            pl.col("cntrycode").cast(str),
            pl.col("numcust"),
            pl.col("totacctbal"))
        noised = pl.read_json(f'../outputs/ap-duckdb-q{query_ind}-customer-{mi}-step3/output.json')
        if index_cols[query_ind]:
            merged_df = orig.join(noised, on=index_cols[query_ind], suffix='_noised')
        else:
            merged_df = orig.join(noised, suffix='_noised', how='cross')
        suffix1 = ''
        suffix2 = '_noised'
        cols_with_suffixes = [col for col in merged_df.columns if suffix1 in col or suffix2 in col]

        base_names = set([col.replace(suffix1, '').replace(suffix2, '') for col in cols_with_suffixes])
        if query_ind == 15:
            base_names = ['total_revenue'] # only look at numeric
        all_outputs = 0.
        for base_name in base_names:
            nulls_exist = False
            null_inds, null_vals = [], []

            orig = base_name + suffix1
            noised = base_name + suffix2
            if orig in merged_df.columns and noised in merged_df.columns:
                rel_errors = []
                for ind in range(len(merged_df[orig])):
                    if query_ind in scale_required:
                        const = 2 # subsampling scaling
                        if query_ind == 1 and not (base_name[:3] == 'sum' or base_name[:5] == 'count'):
                            const = 1 # remove for parts of q1
                    else:
                        const = 1
    #                     print(merged_df[noised][ind])
                    noised_vals = [merged_df[noised][ind][tmp_ind] for tmp_ind in range(
                        len(merged_df[noised][ind])) if merged_df[noised][ind][tmp_ind] is not None]
                    if len(noised_vals) != 1000 and len(noised_vals) != 0:
                        print('reached', query_ind)
                    new = np.average(
                        [100*abs(
                            const*noised_vals[tmp_ind] - merged_df[orig][ind]
                            ) / merged_df[orig][ind] for tmp_ind in range(
                                len(noised_vals))]
                    )
                    if not np.isnan(new):
                        total_error.append(new)
                    rel_errors.append(new)
                merged_df = merged_df.with_columns(pl.Series('pct_diff_' + base_name, rel_errors))
                num_outputs = len(list(merged_df[noised]))
                all_outputs+= num_outputs
                for ind in range(num_outputs):
                    full_list = [x for x in list(merged_df[noised][ind]) if x is not None]
                    if len(full_list) < num_trials:
                        num_nulls += 1
                        nulls_exist = True
                        null_inds.append(ind)
                        null_vals.append(100*(num_trials-len(full_list))/num_trials)
#                         print(query_ind, ind, null_vals)
            null_info[base_name] = (nulls_exist, null_inds, null_vals)
        all_nulls[query_ind] = 100*num_nulls/all_outputs
        avg_errors[query_ind] = np.average(total_error)
        print(query_ind, len(total_error))

#         # Get columns that start with 'pct_diff'
#         pct_diff_cols = [col for col in merged_df.columns if col.startswith('pct_diff')]

#         # Create subplots based on number of pct_diff columns
#         n_cols = len(pct_diff_cols)
#         fig, axes = plt.subplots(1, n_cols, figsize=(5*n_cols, 4))

#         # Handle case where there's only one column (axes won't be a list)
#         if n_cols == 1:
#             axes = [axes]

#         # Create histogram for each column
#         for i, col in enumerate(pct_diff_cols):
#             # Convert to pandas for easier plotting, or use polars plot method
#             values = merged_df[col].to_pandas()

#             axes[i].hist(values, bins=30, alpha=0.7, edgecolor='black')
#             axes[i].set_title(f'Histogram of {col}')
#             axes[i].set_xlabel(col)
#             axes[i].set_ylabel('Frequency')
#             axes[i].grid(True, alpha=0.3)

#         plt.tight_layout()
#         plt.savefig(f'tmp/q{query_ind}_{mi}_{col}_hist.png', facecolor='white', bbox_inches='tight')

#         for base in null_info:
#             (nulls_exist, null_inds, null_vals) = null_info[base]
#             if nulls_exist:
#                 fig, axes = plt.subplots(1, 1, figsize=(5, 4))
#                 # Handle case where there's only one column (axes won't be a list)
#                 if n_cols == 1:
#                     axes = [axes]
#                 axes[0].hist(null_vals, bins=30, alpha=0.7, edgecolor='black')
#                 axes[0].set_title(f'Histogram of nulls for column {base}')
#                 axes[0].set_xlabel('Fraction of nulls (%)')
#                 axes[0].set_ylabel('Frequency')
#                 axes[0].grid(True, alpha=0.3)
#                 plt.savefig(f'tmp/q{query_ind}_{mi}_{base}_nulls.png', facecolor='white', bbox_inches='tight')

1 32
3 0
4 5
5 5
6 1
7 4
8 2
9 175
12 4
13 33
14 1
15 1
17 1
19 1
21 32
22 14


In [26]:
merged_df

cntrycode,numcust,totacctbal,numcust_noised,totacctbal_noised,pct_diff_totacctbal,pct_diff_numcust
str,i64,f64,list[f64],list[f64],f64,f64
"""13""",94,714035.05,"[33.937317, 71.508631, … 69.163039]","[50724.816013, 440978.775174, … 601884.815105]",68.973205,68.516626
"""17""",96,722560.15,"[3.907362, 97.819533, … 30.383614]","[509503.662744, 590296.113815, … 566484.101503]",65.391416,66.69479
"""18""",99,738012.52,"[22.322455, 121.912627, … 46.173817]","[200768.946466, 23306.504846, … 1.1590e6]",63.633958,62.208553
"""23""",93,708285.25,"[65.605016, 60.950854, … 123.327285]","[420105.64541, 393747.436006, … 133587.213973]",62.811913,62.08741
"""29""",85,632693.46,"[90.106648, 44.584117, … 62.374164]","[-123990.720991, 430894.705358, … 376762.958974]",67.942178,60.336616
"""30""",87,646748.02,"[-6.554484, 106.993407, … 25.563236]","[339658.540469, 190784.004115, … 228990.989264]",67.410798,67.192347
"""31""",87,647372.5,"[19.731849, 95.647188, … -8.738575]","[297420.336504, 333912.4853, … -100097.763296]",75.983952,74.386322


In [27]:
all_nulls

{1: 0.0,
 3: 100.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 12: 0.0,
 13: 10.81081081081081,
 14: 0.0,
 15: 0.0,
 17: 0.0,
 19: 0.0,
 21: 31.914893617021278,
 22: 0.0}

In [29]:
avg_errors

{1: 2.5404626070764027,
 3: nan,
 4: 7.947796994535251,
 5: 25.82856629597155,
 6: 3.5772110955356,
 7: 27.382036902648146,
 8: 128.07008478249608,
 9: 23.09227272578598,
 12: 9.29589727474428,
 13: 21.141296570582707,
 14: 7.609964517967354,
 15: 42.68584725608364,
 17: 42.26663021534325,
 19: 84.37800957233543,
 21: 72.67979797478057,
 22: 24.725609167266047}